# Shift in input 


In [0]:
import numpy as np
import torch
import os
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset
import cv2
from PIL import Image

import os
import random
import shutil
import time
import warnings
import sys
import numpy as np
import os
import pandas as pd
import h5py
import logging
import pickle
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.distributed as dist
import torch.optim
import torch.multiprocessing as mp
import torch.utils.data
import torch.utils.data.distributed
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
from google.colab import drive
import matplotlib.pyplot as plt
from torchsummary import summary
from torch.utils.data import DataLoader


In [0]:
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
os.chdir("/content/drive/My Drive/Thesis re-id/triplet-reid-master")

In [0]:
from own_code.loss import TripletLoss
from own_code.triplet_selector import BatchHardTripletSelector
from own_code.batch_sampler import BatchSampler
from own_code.Market1501 import Market1501
#from own_code.Market1501_shift import Market1501
from own_code.backbone import EmbedNetwork
#from own_code.Market1501_no_aug import Market1501
from own_code.optimizer import AdamOptimWrapper
from logger import logger
from models_lpf import *
#import models_lpf.resnet
import models_lpf.resnet
from utils import pdist_np as pdist


In [0]:
# Labels
labels = pd.read_csv('data/market1501_train.csv', names = ['pid', 'fid'], header = None, dtype = str)
labels_query = pd.read_csv('data/market1501_query.csv', names = ['pid', 'fid'], header = None, dtype = str)
labels_test = pd.read_csv('data/market1501_test.csv', names = ['pid', 'fid'], header = None, dtype = str)

### Reading the h5py Files

In [0]:
fileName = 'data_final.h5'
fileName_qt = 'data_qt.h5'

In [0]:
with h5py.File(fileName, "a") as out:
  X_train = np.asarray(out["X_train"])
  #Y_train = np.asarray(out["Y_train"])
  

In [0]:
with h5py.File(fileName_qt, "a") as out:
  X_query = np.asarray(out["X_dev"])
  X_test = np.asarray(out["X_test"])

In [0]:
X_query.shape

(3368, 128, 64, 3)

In [0]:
sum(X_query).shape

(128, 64, 3)

## Embeddings


In [0]:
def create_emb(dataset, fids, net,  store_path, model_num):
  torch.multiprocessing.set_sharing_strategy('file_system')

  ## logging
  FORMAT = '%(levelname)s %(filename)s:%(lineno)d: %(message)s'
  logging.basicConfig(level=logging.INFO, format=FORMAT, stream=sys.stdout)
  logger = logging.getLogger(__name__)
  ## restore model
  logger.info('restoring model')
  model = net
  #model = nn.DataParallel(model)
  model = net.cuda()
  model.module.load_state_dict(torch.load('./res/model{}.pkl'.format(model_num)))
  model = nn.DataParallel(model)
  model.eval()

  ## load gallery dataset
  batchsize = 32
  ds = Market1501(pids_list=list(fids), array=dataset, is_train = False)
  dl = DataLoader(ds, batch_size = batchsize, drop_last = False, num_workers = 4)

  ## embedding samples
  logger.info('start embedding')
  all_iter_nums = len(ds) // batchsize + 1
  embeddings = []
  label_ids = []
  label_cams = []
  for it, (img, lb_id, lb_cam) in enumerate(dl):
    print('\r=======>  processing iter {} / {}'.format(it, all_iter_nums),
            end = '', flush = True)
    label_ids.append(lb_id)
    label_cams.append(lb_cam)
    embds = []
    for im in img:
        im = im.cuda()
        embd = model(im).detach().cpu().numpy()
        embds.append(embd)
    embed = sum(embds) / len(embds)
    embeddings.append(embed)
  print('  ...   completed')

  embeddings = np.vstack(embeddings)
  label_ids = np.hstack(label_ids)
  label_cams = np.hstack(label_cams)

  ## dump results
  logger.info('dump embeddings')
  embd_res = {'embeddings': embeddings, 'label_ids': label_ids, 'label_cams': label_cams}
  with open(store_path, 'wb') as fw:
    pickle.dump(embd_res, fw)

  logger.info('embedding finished')

In [0]:
def create_emb_shift(dataset, fids, net, store_path, model_num):
  torch.multiprocessing.set_sharing_strategy('file_system')

  ## logging
  FORMAT = '%(levelname)s %(filename)s:%(lineno)d: %(message)s'
  logging.basicConfig(level=logging.INFO, format=FORMAT, stream=sys.stdout)
  logger = logging.getLogger(__name__)
  ## restore model
  logger.info('restoring model')
  model = net
  #model = nn.DataParallel(model)
  model = net.cuda()
  model.module.load_state_dict(torch.load('./res/model{}.pkl'.format(model_num)))
  model = nn.DataParallel(model)
  model.eval()

  ## load gallery dataset
  batchsize = 32
  ds = Market1501(pids_list=list(fids), array=dataset, is_train = False)
  dl = DataLoader(ds, batch_size = batchsize, drop_last = False, num_workers = 4)

  ## embedding samples
  logger.info('start embedding')
  all_iter_nums = len(ds) // batchsize + 1
  embeddings = []
  label_ids = []
  label_cams = []
  
  for it, (img, lb_id, lb_cam) in enumerate(dl):
    print('\r=======>  processing iter {} / {}'.format(it, all_iter_nums),
            end = '', flush = True)
    label_ids.append(lb_id)
    label_cams.append(lb_cam)
    embds = []
    #print(img.shape)
    offa = np.random.randint(32,size=2)
    offb = np.random.randint(16,size=2)
    #print(offa,offb)
    for im in img:
      #print(im.shape)
      im = im[:,:,offa[0]:offa[0]+224,offb[0]:offb[0]+112]
      #print(im)
      im = im.cuda()
      embd = model(im).detach().cpu().numpy()
      embds.append(embd)
      #print("end inner")
    embed = sum(embds) / len(embds)
    embeddings.append(embed)
  print('  ...   completed')

  embeddings = np.vstack(embeddings)
  label_ids = np.hstack(label_ids)
  label_cams = np.hstack(label_cams)

  ## dump results
  logger.info('dump embeddings')
  embd_res = {'embeddings': embeddings, 'label_ids': label_ids, 'label_cams': label_cams}
  with open(store_path, 'wb') as fw:
    pickle.dump(embd_res, fw)

  logger.info('embedding finished')


In [0]:
from utils import pdist_np as pdist

In [0]:
def evaluate(test_embs, query_embs, cmc_rank):
    ## logging
    FORMAT = '%(levelname)s %(filename)s:%(lineno)d: %(message)s'
    logging.basicConfig(level=logging.INFO, format=FORMAT, stream=sys.stdout)
    logger = logging.getLogger(__name__)

    ## load embeddings
    logger.info('loading gallery embeddings')
    with open(test_embs, 'rb') as fr:
        gallery_dict = pickle.load(fr)
        emb_gallery, lb_ids_gallery, lb_cams_gallery = gallery_dict['embeddings'], gallery_dict['label_ids'], gallery_dict['label_cams']
    logger.info('loading query embeddings')
    with open(query_embs, 'rb') as fr:
        query_dict = pickle.load(fr)
        emb_query, lb_ids_query, lb_cams_query = query_dict['embeddings'], query_dict['label_ids'], query_dict['label_cams']

    ## compute and clean distance matrix
    dist_mtx = pdist(emb_query, emb_gallery)
    n_q, n_g = dist_mtx.shape
    indices = np.argsort(dist_mtx, axis = 1)
    matches = lb_ids_gallery[indices] == lb_ids_query[:, np.newaxis]
    matches = matches.astype(np.int32)
    all_aps = []
    all_cmcs = []
    logger.info('starting evaluating ...')
    for qidx in tqdm(range(n_q)):
        qpid = lb_ids_query[qidx]
        qcam = lb_cams_query[qidx]

        order = indices[qidx]
        pid_diff = lb_ids_gallery[order] != qpid
        cam_diff = lb_cams_gallery[order] != qcam
        useful = lb_ids_gallery[order] != -1
        keep = np.logical_or(pid_diff, cam_diff)
        keep = np.logical_and(keep, useful)
        match = matches[qidx][keep]

        if not np.any(match): continue

        cmc = match.cumsum()
        cmc[cmc > 1] = 1
        all_cmcs.append(cmc[:cmc_rank])

        num_real = match.sum()
        match_cum = match.cumsum()
        match_cum = [el / (1.0 + i) for i, el in enumerate(match_cum)]
        match_cum = np.array(match_cum) * match
        ap = match_cum.sum() / num_real
        all_aps.append(ap)

    assert len(all_aps) > 0, "NO QUERY MATCHED"
    mAP = sum(all_aps) / len(all_aps)
    all_cmcs = np.array(all_cmcs, dtype = np.float32)
    cmc = np.mean(all_cmcs, axis = 0)

    print('mAP is: {}, cmc is: {}'.format(mAP, cmc))

## Model 1

In [0]:
torch.multiprocessing.set_sharing_strategy('file_system')
if not os.path.exists('./res'): os.makedirs('./res')

In [0]:
model_num = 1
logger.info('setting up backbone model and loss')
net = EmbedNetwork(pretrained_base=True).cuda()
net_1 = nn.DataParallel(net)
summary(net_1, (3,128,64))

setting up backbone model and loss


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 64, 32]           9,408
       BatchNorm2d-2           [-1, 64, 64, 32]             128
              ReLU-3           [-1, 64, 64, 32]               0
         MaxPool2d-4           [-1, 64, 32, 16]               0
            Conv2d-5           [-1, 64, 32, 16]           4,096
       BatchNorm2d-6           [-1, 64, 32, 16]             128
              ReLU-7           [-1, 64, 32, 16]               0
            Conv2d-8           [-1, 64, 32, 16]          36,864
       BatchNorm2d-9           [-1, 64, 32, 16]             128
             ReLU-10           [-1, 64, 32, 16]               0
           Conv2d-11          [-1, 256, 32, 16]          16,384
      BatchNorm2d-12          [-1, 256, 32, 16]             512
           Conv2d-13          [-1, 256, 32, 16]          16,384
      BatchNorm2d-14          [-1, 256,

In [0]:
model_num = 1

In [0]:
#create_emb_shift(dataset = X_query, net = net_1, fids = labels_query.fid, model_num = 1, store_path= "./res/emb_query_shift{}.pkl".format(model_num))
create_emb(dataset = X_test, net = net_1, fids = labels_test.fid, model_num = 1, store_path="./res/emb_test{}.pkl".format(model_num))

=======>  processing iter 616 / 617  ...   completed


In [0]:
test_embs = "./res/emb_test{}.pkl".format(model_num)
query_embs = "./res/emb_query_shift{}.pkl".format(model_num)
cmc_rank = 5
evaluate(test_embs = test_embs, query_embs = query_embs, cmc_rank = cmc_rank)


100%|██████████| 3368/3368 [02:41<00:00, 21.62it/s]


mAP is: 0.6982403809399482, cmc is: [0.8503563  0.89786226 0.9171615  0.932601   0.942696  ]


## model 6

In [0]:
torch.multiprocessing.set_sharing_strategy('file_system')
if not os.path.exists('./res'): os.makedirs('./res')

In [0]:
model_num = 6
filter_size = 3
net = models_lpf.resnet.resnet50(filter_size=filter_size)
net.load_state_dict(torch.load('models_lpf/resnet50_lpf%i.pth.tar'%filter_size)['state_dict'])
model = torch.nn.Sequential(*(list(net.children())[:-2]))

In [0]:
class DenseNormReLU(nn.Module):
    def __init__(self, in_feats, out_feats, *args, **kwargs):
        super(DenseNormReLU, self).__init__(*args, **kwargs)
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.dense = nn.Linear(in_features = in_feats, out_features = out_feats).to(self.device)
        self.bn = nn.BatchNorm1d(out_feats).to(self.device)
        self.relu = nn.ReLU(inplace = True).to(self.device)

    def forward(self, x):
        x = self.dense(x)
        x = self.bn(x)
        x = self.relu(x)
        return x
  
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
fc_head = DenseNormReLU(in_feats = 2048, out_feats = 1024)
embed = nn.Linear(in_features = 1024, out_features = 128).to(device)

In [0]:
class Model(nn.Module):
  def __init__(self):
    super(Model, self).__init__()
    self.base = model
    self.fc_head = fc_head
    self.embed = embed

  def forward(self, x):
    # shape [N, C, H, W]
    x = self.base(x)
    x = F.avg_pool2d(x, x.size()[2:])
    x = x.contiguous().view(-1, 2048 )
    # shape [N, C]
    x = self.fc_head(x)
    x = self.embed(x)

    return x

In [0]:
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Model()
#model = Model().to(device)
model = model.cuda()
net_6 = nn.DataParallel(model)
summary(net_6, (3, 128,64))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 64, 32]           9,408
       BatchNorm2d-2           [-1, 64, 64, 32]             128
              ReLU-3           [-1, 64, 64, 32]               0
         MaxPool2d-4           [-1, 64, 63, 31]               0
   ReflectionPad2d-5           [-1, 64, 65, 33]               0
        Downsample-6           [-1, 64, 32, 16]               0
            Conv2d-7           [-1, 64, 32, 16]           4,096
       BatchNorm2d-8           [-1, 64, 32, 16]             128
              ReLU-9           [-1, 64, 32, 16]               0
           Conv2d-10           [-1, 64, 32, 16]          36,864
      BatchNorm2d-11           [-1, 64, 32, 16]             128
             ReLU-12           [-1, 64, 32, 16]               0
           Conv2d-13          [-1, 256, 32, 16]          16,384
      BatchNorm2d-14          [-1, 256,

In [0]:
model_num = 6

In [0]:
create_emb_shift(dataset = X_query, net = net_6, fids = labels_query.fid, model_num = model_num, store_path= "./res/emb_query_shift{}.pkl".format(model_num))
#create_emb(dataset = X_test, net = net_6, fids = labels_test.fid, model_num = model_num, store_path="./res/emb_test{}.pkl".format(model_num))

=======>  processing iter 105 / 106  ...   completed


In [0]:
test_embs = "./res/emb_test{}.pkl".format(model_num)
query_embs = "./res/emb_query_shift{}.pkl".format(model_num)
cmc_rank = 5
evaluate(test_embs = test_embs, query_embs = query_embs, cmc_rank = cmc_rank)


100%|██████████| 3368/3368 [02:41<00:00, 20.85it/s]


mAP is: 0.7235935030377554, cmc is: [0.8586698  0.90290976 0.9213183  0.93319476 0.9397268 ]


### Model 7
AA model without GAP filter size = 2

In [0]:
torch.multiprocessing.set_sharing_strategy('file_system')
if not os.path.exists('./res'): os.makedirs('./res')

In [0]:
model_num = 7

In [0]:
filter_size = 2
net = models_lpf.resnet.resnet50(filter_size=filter_size)
net.load_state_dict(torch.load('models_lpf/resnet50_lpf%i.pth.tar'%filter_size)['state_dict'])
model = torch.nn.Sequential(*(list(net.children())[:-2]))

In [0]:
class DenseNormReLU(nn.Module):
    def __init__(self, in_feats, out_feats, *args, **kwargs):
        super(DenseNormReLU, self).__init__(*args, **kwargs)
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.dense = nn.Linear(in_features = in_feats, out_features = out_feats).to(self.device)
        self.bn = nn.BatchNorm1d(out_feats).to(self.device)
        self.relu = nn.ReLU(inplace = True).to(self.device)

    def forward(self, x):
        x = self.dense(x)
        x = self.bn(x)
        x = self.relu(x)
        return x
  
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
fc_head = DenseNormReLU(in_feats = 2048, out_feats = 1024)
embedding = nn.Linear(in_features = 1024, out_features = 128).to(device)

In [0]:
class Model(nn.Module):
  def __init__(self):
    super(Model, self).__init__()
    self.base = model
    self.fc_head = fc_head
    self.embedding = embedding

  def forward(self, x):
    # shape [N, C, H, W]
    x = self.base(x)
    x = F.avg_pool2d(x, x.size()[2:])
    x = x.contiguous().view(-1, 2048 )
    # shape [N, C]
    x = self.fc_head(x)
    x = self.embedding(x)

    return x

In [0]:
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Model()
#model = Model().to(device)
model = model.cuda()
net_7 = nn.DataParallel(model)
summary(net_7, (3, 128,64))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 64, 32]           9,408
       BatchNorm2d-2           [-1, 64, 64, 32]             128
              ReLU-3           [-1, 64, 64, 32]               0
         MaxPool2d-4           [-1, 64, 63, 31]               0
   ReflectionPad2d-5           [-1, 64, 64, 32]               0
        Downsample-6           [-1, 64, 32, 16]               0
            Conv2d-7           [-1, 64, 32, 16]           4,096
       BatchNorm2d-8           [-1, 64, 32, 16]             128
              ReLU-9           [-1, 64, 32, 16]               0
           Conv2d-10           [-1, 64, 32, 16]          36,864
      BatchNorm2d-11           [-1, 64, 32, 16]             128
             ReLU-12           [-1, 64, 32, 16]               0
           Conv2d-13          [-1, 256, 32, 16]          16,384
      BatchNorm2d-14          [-1, 256,

In [0]:
model_num = 7

In [0]:
create_emb_shift(dataset = X_query, net = net_7, fids = labels_query.fid, model_num = model_num, store_path= "./res/emb_query_shift{}.pkl".format(model_num))

=======>  processing iter 105 / 106  ...   completed


In [0]:
test_embs = "./res/emb_test{}.pkl".format(model_num)
query_embs = "./res/emb_query_shift{}.pkl".format(model_num)
cmc_rank = 5
evaluate(test_embs = test_embs, query_embs = query_embs, cmc_rank = cmc_rank)


100%|██████████| 3368/3368 [02:41<00:00, 20.50it/s]


mAP is: 0.7252749732281969, cmc is: [0.85807604 0.90825415 0.9245843  0.93557006 0.945962  ]


### Model 8
AA model without GAP, filter size = 5

In [0]:
torch.multiprocessing.set_sharing_strategy('file_system')
if not os.path.exists('./res'): os.makedirs('./res')

In [0]:
model_num = 8

In [0]:
filter_size = 5
net = models_lpf.resnet.resnet50(filter_size=filter_size)
net.load_state_dict(torch.load('models_lpf/resnet50_lpf%i.pth.tar'%filter_size)['state_dict'])
model = torch.nn.Sequential(*(list(net.children())[:-2]))

In [0]:
class DenseNormReLU(nn.Module):
    def __init__(self, in_feats, out_feats, *args, **kwargs):
        super(DenseNormReLU, self).__init__(*args, **kwargs)
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.dense = nn.Linear(in_features = in_feats, out_features = out_feats).to(self.device)
        self.bn = nn.BatchNorm1d(out_feats).to(self.device)
        self.relu = nn.ReLU(inplace = True).to(self.device)

    def forward(self, x):
        x = self.dense(x)
        x = self.bn(x)
        x = self.relu(x)
        return x
  
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
fc_head = DenseNormReLU(in_feats = 2048, out_feats = 1024)
embedding = nn.Linear(in_features = 1024, out_features = 128).to(device)

In [0]:
class Model(nn.Module):
  def __init__(self):
    super(Model, self).__init__()
    self.base = model
    self.fc_head = fc_head
    self.embedding = embedding

  def forward(self, x):
    # shape [N, C, H, W]
    x = self.base(x)
    x = F.avg_pool2d(x, x.size()[2:])
    x = x.contiguous().view(-1, 2048 )
    # shape [N, C]
    x = self.fc_head(x)
    x = self.embedding(x)

    return x

In [0]:
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Model()
#model = Model().to(device)
model = model.cuda()
net_8 = nn.DataParallel(model)
summary(net_8, (3, 128,64))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 64, 32]           9,408
       BatchNorm2d-2           [-1, 64, 64, 32]             128
              ReLU-3           [-1, 64, 64, 32]               0
         MaxPool2d-4           [-1, 64, 63, 31]               0
   ReflectionPad2d-5           [-1, 64, 67, 35]               0
        Downsample-6           [-1, 64, 32, 16]               0
            Conv2d-7           [-1, 64, 32, 16]           4,096
       BatchNorm2d-8           [-1, 64, 32, 16]             128
              ReLU-9           [-1, 64, 32, 16]               0
           Conv2d-10           [-1, 64, 32, 16]          36,864
      BatchNorm2d-11           [-1, 64, 32, 16]             128
             ReLU-12           [-1, 64, 32, 16]               0
           Conv2d-13          [-1, 256, 32, 16]          16,384
      BatchNorm2d-14          [-1, 256,

In [0]:
model_num = 8

In [0]:
create_emb_shift(dataset = X_query, net = net_8, fids = labels_query.fid, model_num = model_num, store_path= "./res/emb_query_shift{}.pkl".format(model_num))

=======>  processing iter 105 / 106  ...   completed


In [0]:
test_embs = "./res/emb_test{}.pkl".format(model_num)
query_embs = "./res/emb_query_shift{}.pkl".format(model_num)
cmc_rank = 5
evaluate(test_embs = test_embs, query_embs = query_embs, cmc_rank = cmc_rank)


100%|██████████| 3368/3368 [02:48<00:00, 20.01it/s]


mAP is: 0.7165804445387892, cmc is: [0.85659146 0.9023159  0.92577195 0.939133   0.9465558 ]


## try with no ten crops

In [0]:
from own_code.Market1501_shift import Market1501


with no ten crop and random shifting for each im


In [0]:
def create_emb_shift_no_crop1(dataset, fids, net, store_path, model_num):
  torch.multiprocessing.set_sharing_strategy('file_system')

  ## logging
  FORMAT = '%(levelname)s %(filename)s:%(lineno)d: %(message)s'
  logging.basicConfig(level=logging.INFO, format=FORMAT, stream=sys.stdout)
  logger = logging.getLogger(__name__)
  ## restore model
  logger.info('restoring model')
  model = net
  #model = nn.DataParallel(model)
  model = net.cuda()
  model.module.load_state_dict(torch.load('./res/model{}.pkl'.format(model_num)))
  model = nn.DataParallel(model)
  model.eval()

  ## load gallery dataset
  batchsize = 32
  ds = Market1501(pids_list=list(fids), array=dataset, is_train = False)
  dl = DataLoader(ds, batch_size = batchsize, drop_last = False, num_workers = 4)

  ## embedding samples
  logger.info('start embedding')
  all_iter_nums = len(ds) // batchsize + 1
  embeddings = []
  label_ids = []
  label_cams = []
  for it, (img, lb_id, lb_cam) in enumerate(dl):
    print('\r=======>  processing iter {} / {}'.format(it, all_iter_nums),
            end = '', flush = True)
    label_ids.append(lb_id)
    label_cams.append(lb_cam)
    embds = []
    for im in img:
      #print(im.shape)
      offa = np.random.randint(32,size=2)
      offb = np.random.randint(16,size=2)
      output0 = model(im[:,offa[0]:offa[0]+224,offb[0]:offb[0]+112])
      im = im.cuda()
      embd = model(im).detach().cpu().numpy()
      embeddings.append(embd)
    #embed = sum(embds) / len(embds)
    #embeddings.append(embed)
  print('  ...   completed')

  embeddings = np.vstack(embeddings)
  label_ids = np.hstack(label_ids)
  label_cams = np.hstack(label_cams)

  ## dump results
  logger.info('dump embeddings')
  embd_res = {'embeddings': embeddings, 'label_ids': label_ids, 'label_cams': label_cams}
  with open(store_path, 'wb') as fw:
    pickle.dump(embd_res, fw)

  logger.info('embedding finished')
